This notebook works by honing in on an XGBClassifier paramterization using Optuna.

Kaggle/your own GPU is reccomended here since we are training hundreds of XGB models.

In [ ]:
#import required libraries
import pandas as pd
import numpy as np
import optuna
from numpy import mean, std
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from optuna import Trial
from optuna.samplers import TPESampler

In [ ]:
#bring in data
raw_training = pd.read_csv("../input/tabular-playground-series-jun-2021/train.csv")
raw_test = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")

#pull out features
X = raw_training.iloc[:, 1:76]
X = np.ascontiguousarray(X)

#pull out responses
y = raw_training.iloc[:, 76]
y = LabelEncoder().fit_transform(y)

Thanks to Adesh Bansode for starter code on optuna

https://medium.com/subex-ai-labs/efficient-hyperparameter-optimization-for-xgboost-model-using-optuna-3ee9a02566b1

In [ ]:
# #define objective function
# def objective(trial: Trial,X,y) -> float:
    
#     #create a data split for validation
#     train_X,test_X,train_y,test_y = train_test_split(X, y, test_size = 0.30, random_state = 101)
    
#     #create a parameter space
#     #Note: I started very general with wide intervals and honed in with repeated studies.
#     #The history dataframe in the next cell is your friend.
#     #Read the table to pare down your intervals after each study.
    
#     param = {                
#                  'learning_rate':trial.suggest_uniform('learning_rate', 0.03, 0.06),
#                  'gamma':trial.suggest_uniform('gamma', .2, .5),
#                  'reg_alpha':trial.suggest_int('reg_alpha', 1, 5),
#                  'reg_lambda':trial.suggest_int('reg_lambda', 1, 7),
#                  'n_estimators':trial.suggest_int('n_estimators', 300, 500),
#                  'colsample_bynode':trial.suggest_uniform('colsample_bynode', .2, .4),
#                  'colsample_bylevel':trial.suggest_uniform('colsample_bylevel', .65, .75),
#                  'subsample':trial.suggest_uniform('subsample', .55, .75),               
#                  'min_child_weight':trial.suggest_int('min_child_weight', 100, 200),
#                  'colsample_bytree':trial.suggest_uniform('colsample_bytree',0.2, .4)
#             }
    
#     #set up the baseline model parameters.
#     #be careful about how high you let max_depth go
#     #this can lead to long training times and overfitting
    
#     model = XGBClassifier(objective='multi:softprob', eval_metric = "mlogloss", num_class = 9,
#                           tree_method = 'gpu_hist', max_depth = 13, use_label_encoder=False, **param)
    
#     #fit to training sample
#     model.fit(train_X, train_y)
    
#     #return the cv score
#     return -cross_val_score(model, test_X, test_y, cv = 3, scoring = 'neg_log_loss').mean()

In [ ]:
# #run an Optuna study
# study = optuna.create_study(direction='minimize', sampler=TPESampler())
# study.optimize(lambda trial : objective(trial, X, y), n_trials = 25)

# #print our best outcome
# print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
# #take a look at the trial runs to pare down parameter intervals
# hist = study.trials_dataframe()
# hist = hist.sort_values(by = ['value'])
# hist

In [ ]:
#here are the parameters from the study
parms = {'learning_rate': 0.03817329673009776, 'gamma': 0.3993428240049768, 'reg_alpha': 3,
         'reg_lambda': 1, 'n_estimators': 334, 'colsample_bynode': 0.2695766080178446,
         'colsample_bylevel': 0.6832712495239914, 'subsample': 0.6999062848890633,
         'min_child_weight': 100, 'colsample_bytree': 0.34663755614898173}

#create the xgb model we will feed to the probabiltiy calibrator
XGBmod = XGBClassifier(objective='multi:softprob',
                         eval_metric = "mlogloss",
                         num_class = 9,
                         tree_method = 'gpu_hist',
                         max_depth = 13,
                         use_label_encoder=False, **parms)

In [ ]:
#take a look at the model perfomance                     
XGBscores = -cross_val_score(XGBmod, X, y, cv = 5, scoring = 'neg_log_loss')
print("Mean log loss: ", XGBscores.mean())
print("Sigma of log loss: ", XGBscores.std())

#make final fit of model
XGBmod.fit(X, y)

In [ ]:
#grab test features
X_test = raw_test.iloc[:,1:76]
X_test = np.ascontiguousarray(X_test)

#predict probabilities on test data
test_pred = XGBmod.predict_proba(X_test)
test_pred = pd.DataFrame(test_pred)
test_pred

In [ ]:
#prepare output and save
output = pd.DataFrame(raw_test.iloc[:,0])

output = output.merge(test_pred, left_index = True, right_index = True)

output.columns = ["id", "Class_1", "Class_2", "Class_3", "Class_4",
                  "Class_5", "Class_6", "Class_7", "Class_8", "Class_9"]

output.to_csv('submission.csv', index=False)